#  aitextgen — Train a GPT-2 Text-Generating Model w/ GPU

by [Max Woolf](https://minimaxir.com)

*Last updated: Jul 5th, 2020*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Colaboratory** using `aitextgen`!

For more about `aitextgen`, you can visit [this GitHub repository](https://github.com/minimaxir/aitextgen) or [read the documentation](https://docs.aitextgen.io/).


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Run the cells below:


In [1]:
!pip install transformers==2.9.1
!pip install pytorch-lightning==0.7.6

!pip install -q aitextgen==0.2.3


import logging
logging.basicConfig(
        format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO
    )

from aitextgen import aitextgen
from aitextgen.colab import mount_gdrive, copy_file_from_gdrive

     |████████████████████████████████| 645kB 8.3MB/s 
     |████████████████████████████████| 1.2MB 14.7MB/s 
     |████████████████████████████████| 890kB 35.4MB/s 
     |████████████████████████████████| 3.8MB 22.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=8b6590f28c63456e8fdfcbf9eeba6a7dbfb9d3551c53f0f3f658dc15ceccf83c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 256kB 9.1MB/s 
     |████████████████████████████████| 829kB 15.1MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=ffb3b62ec010b8231bf06f0019e5ec8cfc30d3e4958cb166901e675b4e9852d3
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
     

01/24/2021 04:07:18 — INFO — transformers.file_utils — PyTorch version 1.7.0+cu101 available.
01/24/2021 04:07:20 — INFO — transformers.file_utils — TensorFlow version 2.4.0 available.


## GPU

Colaboratory uses a Nvidia P4, an Nvidia T4, or an Nvidia P100 GPU. For finetuning GPT-2 124M, any of these GPUs will be fine, but for text generation, a T4 or a P100 is ideal since they have more VRAM.

You can verify which GPU is active by running the cell below. If you want to try for a different GPU, go to **Runtime -> Factory Reset Runtime**.

In [2]:
!nvidia-smi

Sun Jan 24 04:07:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Loading GPT-2

If you're retraining a model on new text, you need to download and load the GPT-2 model into the GPU. 

There are several sizes of GPT-2: currently, aitextgen only works with the smallest one:

* `124M` (default): the "small" model, 500MB on disk.

The next cell downloads it from Google's servers and saves it in the Colaboratory VM. If the model has already been downloaded, running this cell will reload it.

In [3]:
ai = aitextgen(tf_gpt2="124M", to_gpu=True)
# ai = aitextgen(model="minimaxir/reddit", to_gpu=True)

01/24/2021 04:07:20 — INFO — aitextgen — Downloading the 124M GPT-2 TensorFlow weights/config from Google's servers


01/24/2021 04:07:24 — INFO — aitextgen — Converting the 124M GPT-2 TensorFlow weights to PyTorch.



Save PyTorch model to aitextgen/pytorch_model.bin


01/24/2021 04:07:29 — INFO — aitextgen — Loading 124M GPT-2 model from /aitextgen.


Save configuration file to aitextgen/config.json


01/24/2021 04:07:33 — INFO — aitextgen — Using the default GPT-2 Tokenizer.


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [5]:
mount_gdrive()

Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/w3wvHhR.png)

Upload **any smaller text file** (for example, [a text file of Shakespeare plays](https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt)) and update the file name in the cell below, then run the cell.

In [4]:
file_name = "jai.txt"

If your text file is large (>10MB), it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

Additionally, you may want to consider [compressing the dataset to a cache first](https://docs.aitextgen.io/dataset/) on your local computer, then uploading the resulting `dataset_cache.tar.gz` and setting the `file_name`in the previous cell to that.

In [6]:
copy_file_from_gdrive(file_name)

FileNotFoundError: ignored

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2 in aitextgen. It runs for `num_steps`, and a progress bar will appear to show training progress, current loss (the lower the better the model), and average loss (to give a sense on loss trajectory).

The model will be saved every `save_every` steps in `trained_model` by default, and when training completes. If you mounted your Google Drive, the model will _also_ be saved there in a unique folder.

The training might time out after 4ish hours; if you did not mount to Google Drive, make sure you end training and save the results so you don't lose them! (if this happens frequently, you may want to consider using [Colab Pro](https://colab.research.google.com/signup))

Important parameters for `train()`:

- **`line_by_line`**: Set this to `True` if the input text file is a single-column CSV, with one record per row. aitextgen will automatically process it optimally.
- **`from_cache`**: If you compressed your dataset locally (as noted in the previous section) and are using that cache file, set this to `True`.
- **`num_steps`**: Number of steps to train the model for.
- **`generate_every`**: Interval of steps to generate example text from the model; good for qualitatively validating training.
- **`save_every`**: Interval of steps to save the model: the model will be saved in the VM to `/trained_model`.
- **`save_gdrive`**: Set this to `True` to copy the model to a unique folder in your Google Drive, if you have mounted it in the earlier cells

Here are other important parameters for `train()` that are useful but you likely do not need to change.

- **`learning_rate`**: Learning rate of the model training.
- **`batch_size`**: Batch size of the model training; setting it too high will cause the GPU to go OOM.

In [6]:
ai.train(file_name,
         line_by_line=True,
         from_cache=False,
         num_steps=5000,
         generate_every=1000,
         save_every=1000,
         save_gdrive=True,
         learning_rate=1e-4,
         batch_size=2,
         )

01/24/2021 04:12:36 — INFO — aitextgen.TokenDataset — Encoding 29,346 sets of tokens from jai.txt.
GPU available: True, used: True
01/24/2021 04:12:37 — INFO — lightning — GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
01/24/2021 04:12:37 — WARNING — lightning — No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
01/24/2021 04:12:37 — INFO — lightning — CUDA_VISIBLE_DEVICES: [0]


1,000 steps reached: saving model to /trained_model
1,000 steps reached: generating sample texts.


2,000 steps reached: saving model to /trained_model
2,000 steps reached: generating sample texts.
(y)

3,000 steps reached: saving model to /trained_model
3,000 steps reached: generating sample texts.
How was that 816?

4,000 steps reached: saving model to /trained_model
4,000 steps reached: generating sample texts.
I'll ask her to dinner

5,000 steps reached: saving model to /trained_model


01/24/2021 05:22:07 — INFO — aitextgen — Saving trained model pytorch_model.bin to /trained_model


5,000 steps reached: generating sample texts.




You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.


## Load a Trained Model

Running the next cell will copy the `pytorch_model.bin` and the `config.json`file from the specified folder in Google Drive into the Colaboratory VM. (If no `from_folder` is specified, it assumes the two files are located at the root level of your Google Drive)

In [7]:
from_folder = "/content/trained_model"

for file in ["pytorch_model.bin", "config.json"]:
  if from_folder:
    copy_file_from_gdrive(file, from_folder)
  else:
    copy_file_from_gdrive(file)

The next cell will allow you to load the retrained model + metadata necessary to generate text.

In [8]:
ai = aitextgen(model="pytorch_model.bin", config="config.json", to_gpu=True)

01/24/2021 05:23:01 — INFO — aitextgen — Loading GPT-2 model from provided pytorch_model.bin.
01/24/2021 05:23:05 — INFO — aitextgen — Using the default GPT-2 Tokenizer.


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate()` without any parameters generates a single text from the loaded model to the console.

In [13]:
ai.generate(n=10)



END IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT  SEND IT 
I was asleep when he got home today



Lmao





hahaahahaahhah



im in love with you



If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = ai.generate_one()`

You can also pass in a `prompt` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `n`. You can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 50 for `batch_size` to avoid going OOM).

Other optional-but-helpful parameters for `ai.generate()` and friends:

*  **`max_length`**: Number of tokens to generate (default 256, you can generate up to 1024 tokens with GPT-2, but it will be _much_ slower)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)

In [66]:
ai.generate(n=5,
            batch_size=5,
            prompt="the greatest feeling in the world is",
            max_length=256,
            temperature=2,
            top_p=0.9)

the greatest feeling in the world is... our mutual friend samward telling me that no matter what he does, everything he says is OK (except blunder)

the greatest feeling in the world is

the greatest feeling in the world is nah mf

the greatest feeling in the world is... Idk how to put it...

the greatest feeling in the world is



For bulk generation, you can generate a large amount of texts to a file and sort out the samples locally on your computer. The next cell will generate `num_files` files, each with `n` texts and whatever other parameters you would pass to `generate()`. The files can then be downloaded from the Files sidebar!

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
num_files = 5

for _ in range(num_files):
  ai.generate_to_file(n=1000,
                     batch_size=50,
                     prompt="ROMEO:",
                     max_length=256,
                     temperature=1.0,
                     top_p=0.9)

05/18/2020 00:06:16 — INFO — aitextgen — Generating 1,000 texts to ATG_20200518_000616_78407959.txt


05/18/2020 00:08:30 — INFO — aitextgen — Generating 1,000 texts to ATG_20200518_000830_67373043.txt


05/18/2020 00:10:44 — INFO — aitextgen — Generating 1,000 texts to ATG_20200518_001044_88174494.txt


05/18/2020 00:12:58 — INFO — aitextgen — Generating 1,000 texts to ATG_20200518_001258_27768790.txt


05/18/2020 00:15:12 — INFO — aitextgen — Generating 1,000 texts to ATG_20200518_001512_83597962.txt


# LICENSE

MIT License

Copyright (c) 2020 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.